## Osoitteiden perusteella laskettu etäisyys Suomen kaikkiin postinumeroihin



In [1]:
pip install geopandas pandas shapely routingpy numpy

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [155]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point
from routingpy.routers import MapboxOSRM
import numpy as np


# Haetaan dentalin pisteet (poistettu esim. kerrostieto)
osoitteet = pd.read_excel('Dentalin_toimipisteet_2024_12_03.xlsx', sheet_name='Toimipisteet', usecols='A,B:E')
koordinaatit = pd.read_csv('finland-postal-codes-with-polygons.csv')

In [156]:
koordinaatit['postinumero']=koordinaatit.postinumeroalue
koordinaatit['latitude']=koordinaatit.pituusaste
koordinaatit['longitude']=koordinaatit.leveysaste

In [157]:
# Fetch the centre of postal code as lat and lon
df=koordinaatit[['postinumero','longitude','latitude']]

In [158]:
# Concatenate the columns to Full_Address
osoitteet["Formatted_PostalCode"] = osoitteet["Address_PostalCode"].apply(lambda x: f"{x:05d}")

osoitteet["Full_Address"] = (
    osoitteet["Address_Street"] + ", " +
    osoitteet["Formatted_PostalCode"].astype(str) + ", " +
    osoitteet["Address_City"]
)

osoitteet

,Id,Name,Address_Street,Address_PostalCode,Address_City,Formatted_PostalCode,Full_Address
0,76FFBB51-0A7C-480C-945C-055A0A94055E,Terveystalo Kamppi hammaslääkäripalvelut,Jaakonkatu 3,100,Helsinki,00100,"Jaakonkatu 3, 00100, Helsinki"
1,0DD36F02-54F2-4832-9EA6-E8B280699F52,Terveystalo Kamppi hammaslääkäripalvelut (Hiet...,Hietaniemenkatu 6,100,Helsinki,00100,"Hietaniemenkatu 6, 00100, Helsinki"
2,50A917E9-8934-4971-BC7D-B615BA1620BC,Terveystalo Rautatientori Hammaslääkäripalvelut,Kaisaniemenkatu 4,100,Helsinki,00100,"Kaisaniemenkatu 4, 00100, Helsinki"
3,585E027A-71E5-4B0D-A056-D559294B10BF,Helsinki Kamppi Olo-apteekki Graniittitalo,Jaakonkatu 3,100,Helsinki,00100,"Jaakonkatu 3, 00100, Helsinki"
4,49A2F23F-1527-433F-B2BC-318F960E4132,Terveystalo Tilkka Hammaslääkäripalvelut,Mannerheimintie 164 A,300,Helsinki,00300,"Mannerheimintie 164 A, 00300, Helsinki"
...,...,...,...,...,...,...,...
57,AF169E1C-D606-4D0D-A323-7A5ECCFAEA1D,Terveystalo Oulu Hammaslääkäripalvelut Aaria (...,Kirkkokatu 27,90100,Oulu,90100,"Kirkkokatu 27, 90100, Oulu"
58,7EAA49B9-415D-4425-8DA3-30FE77E4ED64,Terveystalo Oulu Hammaslääkäripalvelut (Pateni...,Keskuskatu 7,90800,Oulu,90800,"Keskuskatu 7 , 90800, Oulu"
59,E4834A11-C2DA-4029-9A7E-D87B6DA46BBB,Terveystalo Rovaniemi hammaslääkäripalvelut,Ukkoherrantie 15,96100,Rovaniemi,96100,"Ukkoherrantie 15, 96100, Rovaniemi"
60,F7B33A8C-6830-488B-A8AC-BDB1E92E6434,Attendo Rovaniemen Hammaslaboratorio,Ukkoherrantie 15,96100,Rovaniemi,96100,"Ukkoherrantie 15, 96100, Rovaniemi"


In [3]:
pip install geopy

Note: you may need to restart the kernel to use updated packages.


In [159]:
from geopy.geocoders import Nominatim
from geopy.distance import geodesic

# Initialize geolocator
geolocator = Nominatim(user_agent="address_distance_calculator")

# Test
address_1 = "Jaakonkatu 3, 00100 Helsinki"
address_2 = "Kaisaniemenkatu 4, 00100 Helsinki"

# Get the coordinates of the addresses
location_1 = geolocator.geocode(address_1)
location_2 = geolocator.geocode(address_2)

# Extract latitude and longitude
coords_1 = (location_1.latitude, location_1.longitude)
coords_2 = (location_2.latitude, location_2.longitude)

# Calculate the distance (in kilometers)
distance = geodesic(coords_1, coords_2).kilometers

print(f"The distance between the two addresses is {distance:.2f} km.")


GeocoderUnavailable: HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /search?q=Kaisaniemenkatu+4%2C+00100+Helsinki&format=json&limit=1 (Caused by ReadTimeoutError("HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Read timed out. (read timeout=1)"))

In [148]:
# Add coordinates

# Create a cache dictionary
cache = {}

# Function to get coordinates with caching
def get_coordinates(address):
    if address in cache:
        return cache[address]  # Return cached result
    try:
        location = geolocator.geocode(address)
        if location:
            coords = (location.latitude, location.longitude)
            cache[address] = coords  # Store in cache
            return coords
        else:
            return None
    except Exception as e:
        print(f"Error fetching coordinates for {address}: {e}")
        return None

# Apply the function to get coordinates
osoitteet["Coordinates"] = osoitteet["Full_Address"].apply(get_coordinates)

Error fetching coordinates for Ratatie 11 A, 01300, Vantaa: HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /search?q=Ratatie+11+A%2C+01300%2C+Vantaa&format=json&limit=1 (Caused by ReadTimeoutError("HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Read timed out. (read timeout=1)"))
Error fetching coordinates for Talvialantie 4, 42100, Jämsä: HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /search?q=Talvialantie+4%2C+42100%2C+J%C3%A4ms%C3%A4&format=json&limit=1 (Caused by ReadTimeoutError("HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Read timed out. (read timeout=1)"))
Error fetching coordinates for Keskuskatu 10, 48100, Kotka: HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /search?q=Keskuskatu+10%2C+48100%2C+Kotka&format=json&limit=1 (Caused by ReadTimeoutError("HTTPSConnectionPool(host='nominatim

In [150]:
osoitteet

,Id,Name,Address_Street,Address_PostalCode,Address_City,Formatted_PostalCode,Full_Address,Coordinates
0,76FFBB51-0A7C-480C-945C-055A0A94055E,Terveystalo Kamppi hammaslääkäripalvelut,Jaakonkatu 3,100,Helsinki,00100,"Jaakonkatu 3, 00100, Helsinki","(60.1699441, 24.9317617)"
1,0DD36F02-54F2-4832-9EA6-E8B280699F52,Terveystalo Kamppi hammaslääkäripalvelut (Hiet...,Hietaniemenkatu 6,100,Helsinki,00100,"Hietaniemenkatu 6, 00100, Helsinki","(60.1696846, 24.92477015088607)"
2,50A917E9-8934-4971-BC7D-B615BA1620BC,Terveystalo Rautatientori Hammaslääkäripalvelut,Kaisaniemenkatu 4,100,Helsinki,00100,"Kaisaniemenkatu 4, 00100, Helsinki","(60.1715271, 24.947208)"
3,585E027A-71E5-4B0D-A056-D559294B10BF,Helsinki Kamppi Olo-apteekki Graniittitalo,Jaakonkatu 3,100,Helsinki,00100,"Jaakonkatu 3, 00100, Helsinki","(60.1699441, 24.9317617)"
4,49A2F23F-1527-433F-B2BC-318F960E4132,Terveystalo Tilkka Hammaslääkäripalvelut,Mannerheimintie 164 A,300,Helsinki,00300,"Mannerheimintie 164 A, 00300, Helsinki","(60.19918715, 24.900026914511752)"
...,...,...,...,...,...,...,...,...
57,AF169E1C-D606-4D0D-A323-7A5ECCFAEA1D,Terveystalo Oulu Hammaslääkäripalvelut Aaria (...,Kirkkokatu 27,90100,Oulu,90100,"Kirkkokatu 27, 90100, Oulu","(65.01101415, 25.469833234568775)"
58,7EAA49B9-415D-4425-8DA3-30FE77E4ED64,Terveystalo Oulu Hammaslääkäripalvelut (Pateni...,Keskuskatu 7,90800,Oulu,90800,"Keskuskatu 7 , 90800, Oulu","(65.08004264632426, 25.402716752720192)"
59,E4834A11-C2DA-4029-9A7E-D87B6DA46BBB,Terveystalo Rovaniemi hammaslääkäripalvelut,Ukkoherrantie 15,96100,Rovaniemi,96100,"Ukkoherrantie 15, 96100, Rovaniemi","(66.49904269999999, 25.724338206910502)"
60,F7B33A8C-6830-488B-A8AC-BDB1E92E6434,Attendo Rovaniemen Hammaslaboratorio,Ukkoherrantie 15,96100,Rovaniemi,96100,"Ukkoherrantie 15, 96100, Rovaniemi","(66.49904269999999, 25.724338206910502)"


In [152]:
df

,postinumero,longitude,latitude
0,100,24.926875,60.171899
1,120,24.937339,60.163736
2,130,24.950154,60.164295
3,240,24.924274,60.205685
4,350,24.867968,60.207369
...,...,...,...
3021,99930,28.148605,69.497672
3022,99940,29.099588,69.525595
3023,99950,26.276878,69.409787
3024,99980,26.944535,69.675909


In [153]:
import pandas as pd
from geopy.distance import geodesic
import time

zip_codes = df

# Geocode function to get coordinates for `osoitteet` addresses
from geopy.geocoders import Nominatim

geolocator = Nominatim(user_agent="address_distance_calculator")

# Geocode cache to avoid repeated requests
geocode_cache = {}

def get_coordinates(address):
    if address in geocode_cache:
        return geocode_cache[address]
    try:
        location = geolocator.geocode(address)
        if location:
            coords = (location.latitude, location.longitude)
            geocode_cache[address] = coords
            return coords
    except Exception as e:
        print(f"Error geocoding {address}: {e}")
    return None


# Calculate distances between `osoitteet` and `zip_codes`
distances = []

for _, osoite_row in osoitteet.iterrows():
    osoite_Id = osoite_row['Id']
    osoite_coords = osoite_row['Coordinates']
    osoite_address = osoite_row['Full_Address']
    
    for _, zip_row in zip_codes.iterrows():
        zip_coords = (zip_row['latitude'], zip_row['longitude'])
        postinumero = zip_row['postinumero']
        
        # Calculate distance using geodesic
        distance_km = geodesic(osoite_coords, zip_coords).kilometers
        distances.append({
            'Id': osoite_Id,
            'osoitteet_address': osoite_address,
            'zip_postinumero': postinumero,
            'distance_km': distance_km
        })

# Convert results to DataFrame
distance_df = pd.DataFrame(distances)
distance_df


,Id,osoitteet_address,zip_postinumero,distance_km
0,76FFBB51-0A7C-480C-945C-055A0A94055E,"Jaakonkatu 3, 00100, Helsinki",100.0,0.347899
1,76FFBB51-0A7C-480C-945C-055A0A94055E,"Jaakonkatu 3, 00100, Helsinki",120.0,0.757775
2,76FFBB51-0A7C-480C-945C-055A0A94055E,"Jaakonkatu 3, 00100, Helsinki",130.0,1.199547
3,76FFBB51-0A7C-480C-945C-055A0A94055E,"Jaakonkatu 3, 00100, Helsinki",240.0,4.003681
4,76FFBB51-0A7C-480C-945C-055A0A94055E,"Jaakonkatu 3, 00100, Helsinki",350.0,5.469387
...,...,...,...,...
187607,480286D5-07CA-4D7C-A204-794F14B12171,"Kasarmintie 10, 99600, Sodankylä",99930.0,240.827199
187608,480286D5-07CA-4D7C-A204-794F14B12171,"Kasarmintie 10, 99600, Sodankylä",99940.0,256.762653
187609,480286D5-07CA-4D7C-A204-794F14B12171,"Kasarmintie 10, 99600, Sodankylä",99950.0,222.781585
187610,480286D5-07CA-4D7C-A204-794F14B12171,"Kasarmintie 10, 99600, Sodankylä",99980.0,252.513019


In [154]:
distance_df.to_excel('etäisyysmatriisi.xlsx')
#distance_df.to_excel(distance_matrix.xlsx, index=True)